# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,egilsstadir,65.2653,-14.3948,-2.05,69,75,4.12,IS,1701997233
1,1,chernyshevskiy,63.0128,112.4714,-37.09,100,98,0.67,RU,1701997235
2,2,bredasdorp,-34.5322,20.0403,17.43,79,0,3.34,ZA,1701997236
3,3,grytviken,-54.2811,-36.5092,-0.78,94,100,7.20,GS,1701997238
4,4,port mathurin,-19.6833,63.4167,25.95,74,0,3.05,MU,1701997241


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
# city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles=True, by="City", hover_cols=["Lng", "Lat", "City", "Humidity"], s="Humidity")
cityMap = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles=True, hover_cols=["Lng", "Lat", "City", "Humidity"], s="Humidity", scale=0.8, alpha=0.5, c='City')

# Display the map
cityMap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [101]:
# Narrow down cities that fit criteria and drop any results with null values
filteredCity = city_data_df.loc[(city_data_df["Max Temp"] > 5) & (city_data_df["Max Temp"] < 15) & 
                                (city_data_df["Wind Speed"] < 5) & (city_data_df["Humidity"] < 50)].dropna()

# Display sample data
filteredCity.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,pahrump,36.2083,-115.9839,13.86,44,6,1.34,US,1701997086
39,39,evergreen,39.6333,-105.3172,8.84,29,0,3.58,US,1701997278
52,52,hasaki,35.7333,140.8333,14.05,37,0,2.58,JP,1701997295
120,120,hotan,37.0997,79.9269,6.04,12,100,0.72,CN,1701997363
220,220,bereket,39.2446,55.5154,5.74,43,41,4.11,TM,1701997461


### Step 3: Create a new DataFrame called `hotel_df`.

In [102]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filteredCity[["City", "Country", "Lat", "Lng", "Humidity",]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\Ashley\AppData\Local\Temp\ipykernel_22396\2275380476.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
13,pahrump,US,36.2083,-115.9839,44,
39,evergreen,US,39.6333,-105.3172,29,
52,hasaki,JP,35.7333,140.8333,37,
120,hotan,CN,37.0997,79.9269,12,
220,bereket,TM,39.2446,55.5154,43,
319,tamanrasset,DZ,22.7850,5.5228,19,
320,bangjun,CN,39.9911,117.2628,37,
403,katsuura,JP,35.1333,140.3000,41,
444,harnai,PK,30.1005,67.9374,24,
453,hazro city,PK,33.9091,72.4921,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [105]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key, 
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pahrump - nearest hotel: Holiday Inn Express & Suites
evergreen - nearest hotel: Highland Haven Creekside Inn
hasaki - nearest hotel: 大新旅館
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
bereket - nearest hotel: Şemşadyñ mamasy
tamanrasset - nearest hotel: فندق الأمان
bangjun - nearest hotel: No hotel found
katsuura - nearest hotel: 潮騒の宿
harnai - nearest hotel: No hotel found
hazro city - nearest hotel: No hotel found
nishifukuma - nearest hotel: 割烹旅館 正直亭
shingu - nearest hotel: ホテルニューパレス


,City,Country,Lat,Lng,Humidity,Hotel Name
13,pahrump,US,36.2083,-115.9839,44,Holiday Inn Express & Suites
39,evergreen,US,39.6333,-105.3172,29,Highland Haven Creekside Inn
52,hasaki,JP,35.7333,140.8333,37,大新旅館
120,hotan,CN,37.0997,79.9269,12,Hua Yi International Hotel (accepts foreigners)
220,bereket,TM,39.2446,55.5154,43,Şemşadyñ mamasy
319,tamanrasset,DZ,22.7850,5.5228,19,فندق الأمان
320,bangjun,CN,39.9911,117.2628,37,No hotel found
403,katsuura,JP,35.1333,140.3000,41,潮騒の宿
444,harnai,PK,30.1005,67.9374,24,No hotel found
453,hazro city,PK,33.9091,72.4921,30,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [106]:
%%capture --no-display

# Configure the map plot
hotelMap = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles=True, hover_cols=["City", "Country", "Humidity", "Hotel Name", "Lng", "Lat"], 
                                s="Humidity", scale=0.8, alpha=0.5, c='City', ylim=(-60, 80))

# Display the map
hotelMap

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)